In [1]:
from typing import Optional, Tuple, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os
import tiktoken
from tiktoken.load import load_tiktoken_bpe
from pathlib import Path
from typing import (
    AbstractSet,
    cast,
    Collection,
    Dict,
    Iterator,
    List,
    Literal,
    Sequence,
    TypedDict,
    Union,
)
import json
import numpy as np
from torch.nn.attention import sdpa_kernel, SDPBackend
import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# RoPE

In [3]:
def calculate_angles(theeta, dim, seq_len):
    pos = 1/theeta**(torch.arange(0, dim, 2, device=device).float()/dim)
    t = torch.arange(seq_len, device=device)
    freqs = torch.outer(t, pos)
    unit_vecs = torch.polar(torch.ones_like(freqs), freqs)
    return unit_vecs

def brodcast(unit_vecs, x):
    # print(f"unit_vecs.shape: {unit_vecs.shape}\nX.shape: {x.shape[1], x.shape[-1]}")
    assert unit_vecs.shape == (x.shape[1], x.shape[-1])
    n_dim = x.ndim
    shape = [d if i == 1 or i == n_dim-1 else 1 for i,d in enumerate(x.shape)]
    return unit_vecs.view(*shape)

def RoPE(W_Q, W_K, unit_vecs):
    complex_W_Q = torch.view_as_complex(W_Q.float().reshape(*W_Q.shape[:-1], -1, 2))
    complex_W_K = torch.view_as_complex(W_K.float().reshape(*W_K.shape[:-1], -1, 2))
    # print(complex_W_Q.shape)
    pos = brodcast(unit_vecs, complex_W_K)
    embedded_W_Q = torch.view_as_real(complex_W_Q * pos).float().flatten(3)
    embedded_W_K = torch.view_as_real(complex_W_K * pos).float().flatten(3)
    return embedded_W_Q, embedded_W_K

In [4]:
CONFIGURATIONS = {
  "DIM": 3072,
  "FFN_DIM": 8192,
  "N_LAYERS": 28,
  "N_HEADS": 24,
  "N_KV_HEADS": 8,
  "VOCAB_SIZE": 128256,
  "NORM_EPS": 1e-5,
  "ROPE_THETA": 500000,
  "MAX_BATCH_SIZE": 4,
  "MAX_SEQ_LEN": 6000,
  "N_KV_HEAD_REP": 24 // 8,
  "HEAD_DIM": 128
}

# LAYERS

In [5]:
class Attention(nn.Module):
    def __init__(self, dim, n_heads, head_dim, n_kv_heads, n_kv_heads_reps, max_batch_size, max_seq_len):
        super().__init__()
        self.W_Q = nn.Linear(dim, n_heads * head_dim, bias=False)
        self.W_K = nn.Linear(dim, n_kv_heads * head_dim, bias=False)
        self.W_V = nn.Linear(dim, n_kv_heads * head_dim, bias=False)

        self.register_buffer('CACHE_K', torch.zeros(
            (max_batch_size, max_seq_len, n_kv_heads, head_dim))
        )
        self.register_buffer('CACHE_V', torch.zeros(
            (max_batch_size, max_seq_len, n_kv_heads, head_dim))
        )

        self.wo = nn.Linear(dim, dim)

        self.n_heads = n_heads
        self.head_dim = head_dim
        self.n_kv_heads = n_kv_heads
        self.n_kv_heads_reps = n_kv_heads_reps


    def forward(self,x, freq=None, start_pos=0, mask=None):
        bhz, seq_len, _ = x.shape

        query = self.W_Q(x).view(bhz, seq_len, self.n_heads, self.head_dim)
        key = self.W_K(x).view(bhz, seq_len, self.n_kv_heads, self.head_dim)
        value = self.W_V(x).view(bhz, seq_len, self.n_kv_heads, self.head_dim)

        query, key = RoPE(query, key, freq)

        self.CACHE_K[:bhz, start_pos:start_pos+seq_len] = key
        self.CACHE_V[:bhz, start_pos:start_pos+seq_len] = value

        keys = self.CACHE_K[:bhz, :start_pos+seq_len]
        values = self.CACHE_V[:bhz, :start_pos+seq_len]

        keys = torch.repeat_interleave(input=keys, repeats=self.n_kv_heads_reps, dim=-2)
        values = torch.repeat_interleave(input=values, repeats=self.n_kv_heads_reps, dim=-2)

        queries = query.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        out = F.scaled_dot_product_attention(queries, keys, values, attn_mask=mask)
        out = out.transpose(1,2).contiguous().view(bhz, seq_len, -1)

        return self.wo(out)

In [6]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim, norm_eps):
        super().__init__()
        self.norm_eps = norm_eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.norm_eps)
    
    def forward(self, x):
        out = self._norm(x.float()).type_as(x)
        return out * self.weight

In [7]:
class FeedForward(nn.Module):
    def __init__(self, dim, ffn_dim):
        super().__init__()

        self.w1 = nn.Linear(dim, ffn_dim, bias=False)
        self.w3 = nn.Linear(dim, ffn_dim, bias=False)
        self.w2 = nn.Linear(ffn_dim, dim, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

In [8]:
class Transformer_Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.Attention_Norm = RMSNorm(dim=config["DIM"], norm_eps=config["NORM_EPS"])
        self.FFN_Norm = RMSNorm(dim=config["DIM"], norm_eps=config["NORM_EPS"])
        self.Attention = Attention(dim=config["DIM"],
                                   n_heads=config["N_HEADS"],
                                   head_dim=config["HEAD_DIM"],
                                   n_kv_heads=config["N_KV_HEADS"],
                                   n_kv_heads_reps=config["N_KV_HEAD_REP"],
                                   max_batch_size=config["MAX_BATCH_SIZE"],
                                   max_seq_len=config["MAX_SEQ_LEN"])
        self.FeedForward = FeedForward(dim=config["DIM"],
                                       ffn_dim=config["FFN_DIM"])
    def forward(self, x, freq, start_pos, mask):
        shortcut = x
        x = self.Attention_Norm(x)
        x = self.Attention(x, freq, start_pos, mask)
        x = x + shortcut

        shortcut = x
        x = self.FFN_Norm(x)
        x = self.FeedForward(x)
        x = x + shortcut

        return x

In [9]:
class LLAMA_3(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.tok_embedding = nn.Embedding(config["VOCAB_SIZE"], config["DIM"])
        self.layers = nn.ModuleList()
        for _ in range(config["N_LAYERS"]):
            self.layers.append(Transformer_Block(config))
        self.norm = RMSNorm(config["DIM"], config["NORM_EPS"])
        self.output = nn.Linear(config["DIM"], config["VOCAB_SIZE"], bias=False)

        self.register_buffer(
            'freq',
            calculate_angles(
                config["ROPE_THETA"],
                config["HEAD_DIM"],
                config["MAX_SEQ_LEN"] * 2
            )
        )
    
    def forward(self, tokens, start_pos):
        bhz, seq_len = tokens.shape
        x = self.tok_embedding(tokens)
        freq = self.freq[start_pos : start_pos+seq_len]

        mask = None
        if seq_len > 1:
            mask = torch.full((seq_len, seq_len), float("-inf"), device=tokens.device) 
            mask = torch.triu(mask, diagonal=1)

        for layer in self.layers:
            x = layer(x, freq, start_pos, mask )
        x = self.norm(x)
        x = self.output(x).float()
        
        return x
        

In [10]:
llama = LLAMA_3(config=CONFIGURATIONS).to(device)

In [11]:
total_params = sum(p.numel() for p in llama.parameters())
print(f"Total Number of parameters: {total_params:,}")

Total Number of parameters: 3,606,838,272


In [12]:
total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb/1024} GB")

Total size of the model: 13.436519622802734 GB


In [13]:
def Generate_Text(model, idx, max_tokens, context_size, start_pos):
    for _ in range(max_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.inference_mode():
            logits = model(idx_cond, start_pos)
            logits = logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            next_idx = torch.argmax(probs, dim=-1, keepdim=True)
            idx = torch.cat((idx, next_idx), dim=-1)
    return idx

# Tokenizer

In [14]:
class Tokenizer:
    no_reserved_special_tokens = 256
    pat_str = r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"  # noqa: E501

    def __init__(self, model_path):
        mergeable_ranks = load_tiktoken_bpe(model_path)
        num_base_tokens = len(mergeable_ranks)
        special_tokens = [
            "<|begin_of_text|>",
            "<|end_of_text|>",
            "<|reserved_special_token_0|>",
            "<|reserved_special_token_1|>",
            "<|reserved_special_token_2|>",
            "<|reserved_special_token_3|>",
            "<|start_header_id|>",
            "<|end_header_id|>",
            "<|reserved_special_token_4|>",
            "<|eot_id|>",  # end of turn
        ] + [f"<|reserved_special_token_{i}|>" for i in range(5, self.no_reserved_special_tokens - 5)]
        
        self.special_tokens = {
            token: num_base_tokens + i for i, token in enumerate(special_tokens)
        }

        self.model = tiktoken.Encoding(
            name=Path(model_path).name,
            pat_str=self.pat_str,
            mergeable_ranks=mergeable_ranks,
            special_tokens=self.special_tokens
        )
        self.n_words = self.model.n_vocab
        self.bos_id: int = self.special_tokens["<|begin_of_text|>"]
        self.eos_id: int = self.special_tokens["<|end_of_text|>"]
        self.pad_id: int = -1
        self.stop_tokens = {
            self.special_tokens["<|end_of_text|>"],
            self.special_tokens["<|eot_id|>"],
        }

    def encode(
        self,
        s: str,
        *,
        bos: bool,
        eos: bool,
        allowed_special: Union[Literal["all"], AbstractSet[str]] = set(),
        disallowed_special: Union[Literal["all"], Collection[str]] = (),
        ):
        TIKTOKEN_MAX_ENCODE_CHARS = 400_000
        MAX_NO_WHITESPACES_CHARS = 25_000
        substrs = (
            substr
            for i in range(0, len(s), TIKTOKEN_MAX_ENCODE_CHARS)
            for substr in self._split_whitespaces_or_nonwhitespaces(
                s[i : i + TIKTOKEN_MAX_ENCODE_CHARS], MAX_NO_WHITESPACES_CHARS
            )
        )
        t: List[int] = []
        for substr in substrs:
            t.extend(
                self.model.encode(
                    substr,
                    allowed_special=allowed_special,
                    disallowed_special=disallowed_special,
                )
            )
        if bos:
            t.insert(0, self.bos_id)
        if eos:
            t.append(self.eos_id)
        return t
    def decode(self, t: Sequence[int]) -> str:
        return self.model.decode(cast(List[int], t))

    def _split_whitespaces_or_nonwhitespaces(self,
        s: str, max_consecutive_slice_len: int
    ):
        current_slice_len = 0
        current_slice_is_space = s[0].isspace() if len(s) > 0 else False
        slice_start = 0

        for i in range(len(s)):
            is_now_space = s[i].isspace()

            if current_slice_is_space ^ is_now_space:
                current_slice_len = 1
                current_slice_is_space = is_now_space
            else:
                current_slice_len += 1
                if current_slice_len > max_consecutive_slice_len:
                    yield s[slice_start:i]
                    slice_start = i
                    current_slice_len = 1
        yield s[slice_start:]     

In [15]:
tok = Tokenizer("Weights/3B-instruct//original/tokenizer.model")

In [16]:
z = tok.encode(s="my name is sarthak", bos=False, eos=False)
z = torch.tensor([z], dtype=torch.long, device=device)
d = Generate_Text(llama, z, 5, 10, 0)
d = d.squeeze(dim=0)
tok.decode(d.tolist())

'my name is sarthak历史cede.serializer(cloneγωγή'

In [17]:
from load_gpt_2_355M import load_consolidated_pth_weights, load_llama_weights2

In [18]:
dir = "./Weights/3B-instruct/original/consolidated.00.pth"
params = load_consolidated_pth_weights(dir)

✅ Loaded 255 tensors from ./Weights/3B-instruct/original/consolidated.00.pth


In [19]:
print(list(params.keys())[:])

['tok_embeddings.weight', 'layers.0.attention.wq.weight', 'layers.0.attention.wk.weight', 'layers.0.attention.wv.weight', 'layers.0.attention.wo.weight', 'layers.0.feed_forward.w1.weight', 'layers.0.feed_forward.w3.weight', 'layers.0.feed_forward.w2.weight', 'layers.0.attention_norm.weight', 'layers.0.ffn_norm.weight', 'layers.1.attention.wq.weight', 'layers.1.attention.wk.weight', 'layers.1.attention.wv.weight', 'layers.1.attention.wo.weight', 'layers.1.feed_forward.w1.weight', 'layers.1.feed_forward.w3.weight', 'layers.1.feed_forward.w2.weight', 'layers.1.attention_norm.weight', 'layers.1.ffn_norm.weight', 'layers.2.attention.wq.weight', 'layers.2.attention.wk.weight', 'layers.2.attention.wv.weight', 'layers.2.attention.wo.weight', 'layers.2.feed_forward.w1.weight', 'layers.2.feed_forward.w3.weight', 'layers.2.feed_forward.w2.weight', 'layers.2.attention_norm.weight', 'layers.2.ffn_norm.weight', 'layers.3.attention.wq.weight', 'layers.3.attention.wk.weight', 'layers.3.attention.wv.we

In [20]:
for n, p in list(llama.named_parameters())[:10]:
    print("BEFORE", n, p.mean().item())

BEFORE tok_embedding.weight -3.601663775043562e-05
BEFORE layers.0.Attention_Norm.weight 1.0
BEFORE layers.0.FFN_Norm.weight 1.0
BEFORE layers.0.Attention.W_Q.weight -4.207213805784704e-06
BEFORE layers.0.Attention.W_K.weight 8.162949598045088e-06
BEFORE layers.0.Attention.W_V.weight -6.162651516206097e-06
BEFORE layers.0.Attention.wo.weight -1.2410536101015168e-06
BEFORE layers.0.Attention.wo.bias -0.0001562635152367875
BEFORE layers.0.FeedForward.w1.weight 9.783564109966392e-07
BEFORE layers.0.FeedForward.w3.weight -9.322870937467087e-07


In [21]:
import torch

def weight_injector(my_model, params):
    """
    Inject weights from a LLaMA-like consolidated checkpoint (params)
    into a model whose naming differs from the original LLaMA structure.
    """

    device = next(my_model.parameters()).device
    dtype = next(my_model.parameters()).dtype

    # --- Embeddings ---
    my_model.tok_embedding.weight.data.copy_(
        params["tok_embeddings.weight"].to(device=device, dtype=dtype)
    )

    # --- Layers ---
    num_layers = len(my_model.layers)
    for i in range(num_layers):
        layer = my_model.layers[i]

        # Attention
        layer.Attention.W_Q.weight.data.copy_(
            params[f"layers.{i}.attention.wq.weight"].to(device=device, dtype=dtype)
        )
        layer.Attention.W_K.weight.data.copy_(
            params[f"layers.{i}.attention.wk.weight"].to(device=device, dtype=dtype)
        )
        layer.Attention.W_V.weight.data.copy_(
            params[f"layers.{i}.attention.wv.weight"].to(device=device, dtype=dtype)
        )
        layer.Attention.wo.weight.data.copy_(
            params[f"layers.{i}.attention.wo.weight"].to(device=device, dtype=dtype)
        )

        # Optional bias (only if present)
        if hasattr(layer.Attention.wo, "bias") and f"layers.{i}.attention.wo.bias" in params:
            layer.Attention.wo.bias.data.copy_(
                params[f"layers.{i}.attention.wo.bias"].to(device=device, dtype=dtype)
            )

        # FeedForward
        layer.FeedForward.w1.weight.data.copy_(
            params[f"layers.{i}.feed_forward.w1.weight"].to(device=device, dtype=dtype)
        )
        layer.FeedForward.w3.weight.data.copy_(
            params[f"layers.{i}.feed_forward.w3.weight"].to(device=device, dtype=dtype)
        )
        layer.FeedForward.w2.weight.data.copy_(
            params[f"layers.{i}.feed_forward.w2.weight"].to(device=device, dtype=dtype)
        )

        # Norms
        layer.Attention_Norm.weight.data.copy_(
            params[f"layers.{i}.attention_norm.weight"].to(device=device, dtype=dtype)
        )
        layer.FFN_Norm.weight.data.copy_(
            params[f"layers.{i}.ffn_norm.weight"].to(device=device, dtype=dtype)
        )

    # --- Final Norm ---
    my_model.norm.weight.data.copy_(
        params["norm.weight"].to(device=device, dtype=dtype)
    )

    # --- Output projection ---
    out_w = params["output.weight"]
    if my_model.output.weight.shape == out_w.shape:
        my_model.output.weight.data.copy_(out_w.to(device=device, dtype=dtype))
    elif my_model.output.weight.shape[::-1] == out_w.shape:
        my_model.output.weight.data.copy_(out_w.T.to(device=device, dtype=dtype))
    else:
        raise ValueError(f"Output weight shape mismatch: model={my_model.output.weight.shape}, ckpt={out_w.shape}")

    print("✅ All weights successfully injected into your model!")


In [22]:
weight_injector(llama, params)

✅ All weights successfully injected into your model!


In [23]:
def generate(model, idx, context_len, max_new_tok, top_k, temp=0.1, eos_id=None):
    model.eval()
    
    for tok_no in range(max_new_tok):
        if tok_no == 0:
            idx_cond = idx[:, -context_len:] if idx.shape[1] > context_len else idx
            start_pos = 0
        else:
            idx_cond = idx[:, -1:]
            start_pos = idx.shape[1] - 1
        
        print(f"\rToken {tok_no}: seq_len={idx_cond.shape[1]}, start_pos={start_pos}", end="")
        
        with torch.inference_mode():
            logits = model(idx_cond, start_pos)
        
        logits = logits[:, -1, :]
        
        if top_k > 0:
            top_k_logits, _ = torch.topk(logits, top_k)
            min_val = top_k_logits[:, -1:]
            logits = torch.where(
                logits < min_val,
                torch.tensor(float("-inf"), device=logits.device),
                logits
            )
        
        if temp > 0.0:
            logits = logits / temp
        
        probs = torch.softmax(logits, dim=-1)
        preds = torch.multinomial(probs, 1)
        
        if eos_id is not None and preds.item() == eos_id:
            print(f"EOS token reached at {tok_no}")
            break
        
        idx = torch.cat([idx, preds], dim=1)
    
    return idx

In [24]:
def format_and_tokenize(system_prompt, user_prompt, context=None, device="cuda"):
    if context:
        user_prompt = f"Use the following context to answer:\n{context.strip()}\n\nQuestion: {user_prompt.strip()}"

    prompt = (
        "<|start_header_id|>system<|end_header_id|>\n"
        f"{system_prompt.strip()}\n"
        "<|eot_id|>\n"
        "<|start_header_id|>user<|end_header_id|>\n"
        f"{user_prompt.strip()}\n"
        "<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>\n"
    )

    input_ids = tok.encode(prompt, bos=True, eos=False)
    return torch.tensor([input_ids], device=device)


In [25]:
system_prompt = "You are a concise and factual AI assistant. When context is provided, summarize it in your own words rather than copying."
user_prompt = "What is sarthaks eConnect?"
context = "Sarthak’s eConnect is an educational platform designed to support students in preparing for competitive and academic exams in India. It provides a wide range of study resources, including notes, sample papers, quizzes, mock tests, and previous year question papers for exams like JEE, NEET, UPSC, SSC, and state boards.The platform also features an active Q&A community, where students, teachers, and experts can interact—asking and answering academic or exam-related questions. It encourages collaborative learning, doubt-solving, and knowledge sharing. Additionally, users can access daily practice questions, current affairs updates, and topic-wise discussions to enhance their preparation and conceptual understanding."
idx = format_and_tokenize(system_prompt, user_prompt, context)

In [26]:
idx.shape

torch.Size([1, 233])

In [27]:
print("=" * 60)
print("DEVICE DIAGNOSTICS:")
print(f"Model weights device: {next(llama.parameters()).device}")
print(f"Layer 0 KV cache K: {llama.layers[0].Attention.CACHE_K.device}")
print(f"Layer 0 KV cache V: {llama.layers[0].Attention.CACHE_V.device}")
print(f"RoPE freqs device: {llama.freq.device}")
print(f"Input tokens device: {idx.device}")
print("=" * 60)

DEVICE DIAGNOSTICS:
Model weights device: cuda:0
Layer 0 KV cache K: cuda:0
Layer 0 KV cache V: cuda:0
RoPE freqs device: cuda:0
Input tokens device: cuda:0


In [28]:
out = generate(llama, idx, context_len=2048, max_new_tok=50, top_k=10, temp=0.1)

Token 49: seq_len=1, start_pos=281

# Decoding Block

In [36]:
def extract_assistant_output(out, tokenizer):
    out = out.squeeze()
    text = tokenizer.decode(out.tolist())
    if "<|start_header_id|>assistant<|end_header_id|>" in text:
        text = text.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    return text.split("<|eot_id|>")[0].strip()

In [37]:
assistant_response = extract_assistant_output(out, tok)
print(assistant_response)

Sarthak's eConnect is an educational platform designed to support students in preparing for competitive and academic exams in India. It provides a wide range of study resources, including notes, sample papers, quizzes, mock tests, and previous year question papers for
